
#Mouachiq Abdelkarim
#Big Textual Data Analytics


In [ ]:
#install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#download spark3.5.4
!wget -q https://archive.apache.org/dist/spark/spark-3.5.4/spark-3.5.4-bin-hadoop3.tgz
#!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
#unzip it
!tar xf spark-3.5.4-bin-hadoop3.tgz
#!tar xf spark-3.2.1-bin-hadoop3.2.tgz
# install findspark
!pip install -q findsparkimport o
import os
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"]="/content/spark-3.5.4-bin-hadoop3"
import sys
!pip install pyspark



^C

gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now
ERROR: Could not find a version that satisfies the requirement findsparkimport (from versions: none)
ERROR: No matching distribution found for findsparkimport


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.functions import *
print(pyspark.__version__)

spark = SparkSession.builder.master("local[*]").appName("TextAnalytics").getOrCreate()
sc=spark.sparkContext

3.5.4


FileNotFoundError: [Errno 2] No such file or directory: '/content/spark-3.5.4-bin-hadoop3/./bin/spark-submit'

In [ ]:
from pyspark import SparkFiles
import string

url_RomeoJuliet= "https://www.gutenberg.org/cache/epub/1112/pg1112.txt"
url_Hamlet= "https://www.gutenberg.org/files/1524/1524-0.txt"
url_RichardII="https://www.gutenberg.org/cache/epub/1776/pg1776.txt"

spark.sparkContext.addFile(url_Hamlet)
spark.sparkContext.addFile(url_RomeoJuliet)
spark.sparkContext.addFile(url_RichardII)


shakespeare1_rdd=sc.textFile(SparkFiles.get("pg1112.txt"))
shakespeare1_rdd.take(100)

shakespeare2_rdd=sc.textFile(SparkFiles.get("1524-0.txt"))
shakespeare2_rdd.take(100)

shakespeare3_rdd=sc.textFile(SparkFiles.get("pg1776.txt"))
shakespeare3_rdd.take(100)

def lower_clean_str(str):
  return str.translate(str.maketrans('','',string.punctuation)).lower()


shakespeare1_rdd_clean=shakespeare1_rdd.map(lower_clean_str)
shakespeare2_rdd_clean=shakespeare2_rdd.map(lower_clean_str)
shakespeare3_rdd_clean=shakespeare3_rdd.map(lower_clean_str)
shakespeare1_rdd_clean.take(30)
shakespeare2_rdd_clean.take(30)



In [ ]:
import re

rdd1_count=shakespeare1_rdd_clean.flatMap(lambda x : x.split(" "))\
.filter(lambda x: re.match('[a-z]+',x))\
.map(lambda x :(x,1))\
.reduceByKey(lambda x,y:x+y)
rdd1_count.take(30)

rdd2_count=shakespeare2_rdd_clean.flatMap(lambda x : x.split(" "))\
.filter(lambda x: re.match('[a-z]+',x))\
.map(lambda x :(x,1))\
.reduceByKey(lambda x,y:x+y)
rdd2_count.take(30)


rdd3_count=shakespeare3_rdd_clean.flatMap(lambda x : x.split(" "))\
.filter(lambda x: re.match('[a-z]+',x))\
.map(lambda x :(x,1))\
.reduceByKey(lambda x,y:x+y)
rdd3_count.take(30)



In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords= stopwords.words('english')

rdd_stopwords = sc.parallelize(stopwords)

rdd1stopwords= rdd1_count.filter(lambda x : x[0] in stopwords)
rdd1stopwords.take(20)

rdd2stopwords= rdd2_count.filter(lambda x : x[0] in stopwords)
rdd2stopwords.take(20)

rdd3stopwords= rdd3_count.filter(lambda x : x[0] in stopwords)
rdd3stopwords.take(20)


allstopwords=rdd1stopwords.union(rdd2stopwords).union(rdd3stopwords)\
.reduceByKey(lambda x,y:x+y)\
.map( lambda x:(x[1],x[0]))\
.sortByKey(False)
allstopwords.take(20)


df = spark.createDataFrame(allstopwords,['Frequency','Words'])
df.show(n=40)

!rm -rf "./stopwords.csv"
df.write.csv("./stopwords.csv",sep=',',header=True)



In [ ]:
rdd1nostopwords= rdd1_count.filter(lambda x : x[0] not in stopwords).distinct()
rdd2nostopwords= rdd2_count.filter(lambda x : x[0] not in stopwords).distinct()
rdd3nostopwords= rdd3_count.filter(lambda x : x[0] not in stopwords).distinct()

rdd1_total=rdd1nostopwords.map(lambda x : (x[0], "pg1112.txt"))
rdd2_total=rdd2nostopwords.map(lambda x : (x[0], "1524-0.txt"))
rdd3_total=rdd3nostopwords.map(lambda x : (x[0], "pg1776.txt"))

allnostopwords=rdd1_total.union(rdd2_total).union(rdd3_total)\
.groupByKey()\
.map(lambda x : (x[0],list(x[1])))\
.zipWithIndex()\
.map(lambda x: (x[1],x[0][0],x[0][1]))

allnostopwords.take(100)

[(0, 'world', ['pg1112.txt', '1524-0.txt', 'pg1776.txt']),
 (1, 'restrictions', ['pg1112.txt', '1524-0.txt', 'pg1776.txt']),
 (2, 'whatsoever', ['pg1112.txt', '1524-0.txt', 'pg1776.txt']),
 (3, 'reuse', ['pg1112.txt', '1524-0.txt', 'pg1776.txt']),
 (4, 'updated', ['pg1112.txt', '1524-0.txt', 'pg1776.txt']),
 (5, 'stand', ['pg1112.txt', '1524-0.txt', 'pg1776.txt']),
 (6, 'shewes', ['pg1112.txt']),
 (7, 'cut', ['pg1112.txt', '1524-0.txt', 'pg1776.txt']),
 (8, 'well', ['pg1112.txt', '1524-0.txt', 'pg1776.txt']),
 (9, 'seruingmen', ['pg1112.txt']),
 (10, 'backe', ['pg1112.txt']),
 (11, 'begin', ['pg1112.txt', '1524-0.txt', 'pg1776.txt']),
 (12, 'gr', ['pg1112.txt']),
 (13, 'nay', ['pg1112.txt', '1524-0.txt', 'pg1776.txt']),
 (14, 'bite', ['pg1112.txt', 'pg1776.txt']),
 (15, 'remember', ['pg1112.txt', '1524-0.txt', 'pg1776.txt']),
 (16, 'fight', ['pg1112.txt', '1524-0.txt', 'pg1776.txt']),
 (17, 'part', ['pg1112.txt', '1524-0.txt', 'pg1776.txt']),
 (18, 'three', ['pg1112.txt', '1524-0.txt',

In [ ]:

nb_rdd1 = rdd1nostopwords.map(lambda x: (x[0], f"RomeoJuliet#{x[1]}"))
nb_rdd2 = rdd2nostopwords.map(lambda x: (x[0], f"Hamlet#{x[1]}"))
nb_rdd3 = rdd3nostopwords.map(lambda x: (x[0], f"RichardII#{x[1]}"))

union= nb_rdd1.union(nb_rdd2).union(nb_rdd3)
reunion=union.groupByKey().map(lambda x : (x[0],list(x[1])))\
.zipWithIndex()\
.map(lambda x: (x[1],x[0][0],x[0][1]))
reunion.take(20)



NameError: name 'rdd1nostopwords' is not defined